# APIs

APIs são mecanismos que permitem que dois componentes de software se comuniquem usando um conjunto de definições e protocolos. Por exemplo, o sistema de software do instituto meteorológico contém dados meteorológicos diários. O aplicativo meteorológico em seu telefone “fala” com este sistema por meio de APIs e mostra atualizações meteorológicas diárias no telefone.

API significa Application Programming Interface (Interface de Programação de Aplicação). No contexto de APIs, a palavra Aplicação refere-se a qualquer software com uma função distinta. A interface pode ser pensada como um contrato de serviço entre duas aplicações. Esse contrato define como as duas se comunicam usando solicitações e respostas. A documentação de suas respectivas APIs contém informações sobre como os desenvolvedores devem estruturar essas solicitações e respostas.

![image.png](images/api.jpeg)

`client`: O cliente

`api`: O garçom

`o serviço (banco de dados, web application, twitter, spotify)`: O pizzaiolo

# Web APIs

## Some Examples

### API endpoints

**NASA API**: http://api.open-notify.org/
- http://api.open-notify.org/iss-now.json
    - Documentation: http://open-notify.org/Open-Notify-API/ISS-Location-Now/

**Meme API**

- http://apimeme.com/?ref=apilist.fun
- http://apimeme.com/meme?meme=Doge&top=Analista&bottom=de_Dados
    

**Public APIs**

- https://github.com/public-apis/public-apis

## Acessando APIs via Requests

`Requests` é uma biblioteca Python que nos permite interagir com servidores [HTTP](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol). Essa biblioteca pode ser utilizada para baixar páginas, arquivos e **interagir com APIs**.



[Documentação - requests](https://requests.readthedocs.io)

In [ ]:
! pip3 install requests

In [1]:
import requests, time

### Utilizando `requests`

Vamos começar acessando uma API simples que nos permite extrair, a cada 5s, a posição da [ISS](https://en.wikipedia.org/wiki/International_Space_Station).

O primeiro passo é descobrir o `endpoint` desta API. O `endpoint` é o URL através do qual acessamos uma API!

In [2]:
endpoint = 'http://api.open-notify.org/iss-now.json'

Com nosso URL em mãos, podemos utilizar a biblioteca `requests` para extrair a informação da API:

In [3]:
response = requests.get(endpoint)

O que é o objeto que guardamos na variável `response`?

In [4]:
response

<Response [200]>

O objeto é do tipo `response` e ele contém o código HTTP associado ao nosso request:

![image.png](images/http_status_codes.jpeg)

Com este código podemos saber se a nossa comunicação com o servidor de certo ou não (e porque não deu certo).

https://www.tutorialspoint.com/http/http_status_codes.htm

Para extrair a informação que a API nos retornou (no caso de um sucesso), podemos utilizar o método `.json()`. 

In [6]:
response.json()

{'iss_position': {'longitude': '147.2133', 'latitude': '-0.1551'},
 'message': 'success',
 'timestamp': 1678823216}

Como nosso *request* teve sucesso (status 200), o método `.json()` nos retorna o resultado como um dicionário ou uma lista:

In [7]:
resultado = response.json()
type(resultado)

dict

#### Erros de HTTP

Um problema comum ao construirmos scripts que acessam API é a tratativa de erros HTTP. Caso a API não retorne o que esperamos as etapas subsequentes de nosso programa podem falhar. As tratativas de erro HTTP são fundamentais em qualquer programa que tenha extraia informações (seja de uma API, seja via web-scrapping).

Vamos simular um erro tentando acessar um endpoint inexistente:

In [8]:
response = requests.get('http://api.open-notify.org/coisas_aleatória.json')
response

<Response [404]>

Através do objeto `Response` já vemos que o status da execução não é de sucesso: 404 indica um **Client Error**. Podemos utilizar o atributo `.status_code` deste objeto para extrair o código de status diretamente:

In [9]:
response.status_code

404

Poderíamos tratar os diferentes códigos de status através de condicionais. No entanto, como são muitos códigos diferentes e esta é uma tarefa muito comum, o objeto `Response` contém um método que realiza essa tratativa: `.raise_for_status()`

In [10]:
response.raise_for_status()

HTTPError: 404 Client Error: Not Found for url: http://api.open-notify.org/coisas_aleat%C3%B3ria.json

O método `.raise_for_status()` *levanta* um erro toda vez que um código que não seja sucesso é retornado. Sendo assim, podemos utilizar um `try:` e `except:` para tratar este `HTTPError`!

In [11]:
from requests import HTTPError

try:
    response = requests.get('http://api.open-notify.org/coisas_aleatória.json')
    response.raise_for_status()
    resultado = response.json()
except HTTPError as e:
    print(f'Falha na comunicação com o servidor:\n {e}')
    resultado = None

print(resultado)

Falha na comunicação com o servidor:
 404 Client Error: Not Found for url: http://api.open-notify.org/coisas_aleat%C3%B3ria.json
None


Vamos comparar o resultado do bloco acima utilizando o *endpoint* real da API:

In [12]:
try:
    response = requests.get('http://api.open-notify.org/iss-now.json')
    response.raise_for_status()
    resultado = response.json()
except HTTPError as e:
    print(f'Falha na comunicação com o servidor:\n {e}')
    resultado = None

print(resultado)

{'iss_position': {'longitude': '149.4793', 'latitude': '-3.3486'}, 'message': 'success', 'timestamp': 1678823279}


## Utilizando APIs I

Até agora vimos como podemos acessar, extrair informações e tratar erros de uma API através de métodos e funções fundamentais. Vamos utilizar este conhecimento para construir um `DataFrame` com as posições da ISS.

In [13]:
from datetime import datetime
import pandas as pd

response = requests.get('http://api.open-notify.org/iss-now.json')
results = response.json()

A primeira etapa de qualquer construção envolvendo uma ou mais APIs é a exploração do objeto retornado.

In [14]:
type(results)

dict

A API da ISS retorna um dicionário: vamos analisar como este dicionário está estruturado:

In [15]:
results.keys()

dict_keys(['iss_position', 'message', 'timestamp'])

Vamos investigar como as informações estão estruturadas dentro destas chaves:

In [16]:
for chave in results.keys():
    print(f"{chave}: {type(results[chave])}")

iss_position: <class 'dict'>
message: <class 'str'>
timestamp: <class 'int'>


As chaves `message` e `timestamp` são tipos simples (`str` e `int` respectivamente). Já a chave `iss_position` é outro dicionário:

In [17]:
results['iss_position']

{'longitude': '149.6957', 'latitude': '-3.6525'}

O resultado desta API é muito simples - temos 3 chaves, sendo que 2 são de interesse para nós:

1. `iss_position` contém duas chaves, `latitude` e `longitude`, contendo a posição da ISS;
1. `timestamp` contém o **timestamp** associado a esta posição.

Primeiro, vamos utilizar a função `fromtimestamp()` do módulo `datetime` para converter nosso `timestamp` em um objeto `DateTime`:

In [18]:
data_hora = datetime.fromtimestamp(results['timestamp'])
print(data_hora)

2023-03-14 16:48:05


Agora que vimos como transformar este resultado, podemos construir um loop para criar uma tabela com a posição da ISS a cada 10s, por 1 minuto:

In [19]:
lat = []
long = []
data_hora = []

for i in range(6):
    response = requests.get('http://api.open-notify.org/iss-now.json')
    results = response.json()

    lat.append(float(results['iss_position']['latitude']))
    long.append(float(results['iss_position']['longitude']))
    data_hora.append(datetime.fromtimestamp(results['timestamp']))

    time.sleep(10)

Agora vamos construir nosso `DataFrame` a partir das listas que criamos em nosso loop:

In [20]:
tb_iss = pd.DataFrame({'lat': lat, 'long':long, 'time':data_hora})

In [21]:
tb_iss

,lat,long,time
0,-4.0069,149.9484,2023-03-14 16:48:12
1,-4.5383,150.3278,2023-03-14 16:48:23
2,-5.0695,150.7079,2023-03-14 16:48:33
3,-5.6003,151.0886,2023-03-14 16:48:44
4,-6.1055,151.4519,2023-03-14 16:48:54
5,-6.6358,151.8342,2023-03-14 16:49:04


In [22]:
tb_iss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   lat     6 non-null      float64       
 1   long    6 non-null      float64       
 2   time    6 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(2)
memory usage: 272.0 bytes


## Utilizando APIs II

A API da ISS é extremamente rudimentar: ela não possui nem autenticação nem parâmetros, simplesmente retorna a posição atual da ISS. Muitas APIs são mais complexas do que isso, necessitando de autenticação e possibilitando a utilização de parâmetros:

1. **Autenticação**: muitas APIs precisam que o seu programa se autentique, ou seja, se identifique. Muitas vezes isso é feito através do site que disponibiliza a API e requer a criação de um usuário;
1. **Parâmetros**: parâmetros são informações que passamos para a API utilizadas para determinar que informação será retornada.

No próximo exemplo utilizaremos a [WeatherAPI](https://www.weatherapi.com). Esta API retorna informações climáticas para um dado local. Ela é autenticada (através de um *token*) e necessita de uma parametrização (a latitude e longitude do local para o qual queremos extrair as informações).

Primeiro vamos criar especificar todos os parâmetros necessários para a consulta à esta API:

In [23]:
token = 'bfef1b47e3b8462199b191725221801'
endpoint = 'http://api.weatherapi.com/v1/current.json'

header = {'key' : token}
query = {'q' : '-21.996160,-47.426420'}

Utilizamos dois dicionários para especificar tanto a autenticação quanto a parametrização de nosso request:

1. `header`: é um dicionário que contém apenas uma chave, `key`, na qual guardamos nosso token identificador;
1. `query`: é um dicionário que contém apenas uma chave, `q`, na qual guardamos um string de latitude e longitude.

O nome das chaves foi determinado lendo a documentação da API: https://www.weatherapi.com/docs/.

Agora vamos utilizar nossos dicionários e a função `request()` para extrair as informações metereológicas da latitude **-21,996160** e longitude **-47,426420**

In [24]:
response = requests.request('GET', endpoint,
                            headers=header, 
                            params=query)
results = response.json()

Utilizamos o argumento `headers =` para passar ao `request()` nossos parâmetros de autenticação, e o argumento `params =` para passar os parâmetros de busca.

In [25]:
results

{'location': {'name': 'Pirassununga',
  'region': 'Sao Paulo',
  'country': 'Brazil',
  'lat': -22.0,
  'lon': -47.43,
  'tz_id': 'America/Sao_Paulo',
  'localtime_epoch': 1678823368,
  'localtime': '2023-03-14 16:49'},
 'current': {'last_updated_epoch': 1678823100,
  'last_updated': '2023-03-14 16:45',
  'temp_c': 23.0,
  'temp_f': 73.4,
  'is_day': 1,
  'condition': {'text': 'Light rain',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/296.png',
   'code': 1183},
  'wind_mph': 3.8,
  'wind_kph': 6.1,
  'wind_degree': 170,
  'wind_dir': 'S',
  'pressure_mb': 1016.0,
  'pressure_in': 30.0,
  'precip_mm': 1.8,
  'precip_in': 0.07,
  'humidity': 89,
  'cloud': 75,
  'feelslike_c': 25.0,
  'feelslike_f': 77.0,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 6.0,
  'gust_mph': 5.1,
  'gust_kph': 8.3}}

Esta API é muito mais complexa que a API da ISS: não só pela autenticação e parametrização, mas também pela complexidade dos resultados retornados.

Primeiro, vamos construir uma aplicação simples cujo objetivo é guardar apenas as informações temperatura, humidade, pressão do ar e velocidade do vento. Para isso precisamos investigar o resultado retornado e determinar quais chaves iremos guardar:

In [26]:
# EXERCICIO
# Extrair as informações de temperatura, humidade, pressão do ar e velocidade do vento 
# para a lista de latitudes e longitudes abaixo.
lista_latlong = [
    '-23.57,-46.69', 
    '-1.34,-48.60', 
    '-7.99,-34.87', 
    '6.51,3.39',
    '33.69,-84.44', 
    '69.00, 33.13'
]

### Utilizando `json_normalize()`

No exercicio acima, guardamos apenas um sub-conjunto das informações extraídas. No entanto, muitas vezes é mais prático guardar todas as informações para depois filtra-la.

Para simplificar este processo podemos utilizar a função `json_normalize()` da biblioteca Pandas.

In [27]:
# Verificar variável response.json()
token = 'bfef1b47e3b8462199b191725221801'

endpoint = 'http://api.weatherapi.com/v1/current.json'
header = {'key' : token}
query = {'q' : '-21.99,-47.42'}


response = requests.request('GET', endpoint, headers=header, params=query)
results = response.json()
print(results)

{'location': {'name': 'Pirassununga', 'region': 'Sao Paulo', 'country': 'Brazil', 'lat': -21.99, 'lon': -47.42, 'tz_id': 'America/Sao_Paulo', 'localtime_epoch': 1678823371, 'localtime': '2023-03-14 16:49'}, 'current': {'last_updated_epoch': 1678823100, 'last_updated': '2023-03-14 16:45', 'temp_c': 23.0, 'temp_f': 73.4, 'is_day': 1, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x64/day/296.png', 'code': 1183}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 170, 'wind_dir': 'S', 'pressure_mb': 1016.0, 'pressure_in': 30.0, 'precip_mm': 1.8, 'precip_in': 0.07, 'humidity': 89, 'cloud': 75, 'feelslike_c': 25.0, 'feelslike_f': 77.0, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 6.0, 'gust_mph': 5.1, 'gust_kph': 8.3}}


Primeiro vamos construir um loop para percorrer uma lista de latitudes e longitudes e armazenar os dicionários de resultados em uma lista:

In [28]:
lista_latlong = [
    '-23.57,-46.69', 
    '-1.34,-48.60', 
    '-7.99,-34.87', 
    '6.51,3.39',
    '33.69,-84.44', 
    '69.00, 33.13'
]

In [29]:
token = 'bfef1b47e3b8462199b191725221801'
endpoint = 'http://api.weatherapi.com/v1/current.json'
header = {'key' : token}
lista_resultados = []
for latlong in lista_latlong:
    query = {'q' : latlong}
    response = requests.request('GET', endpoint, headers=header, params=query)
    results = response.json()
    lista_resultados.append(results)

A variável `lista_resultados` é uma `list` de `dicts`: cada elemento desta lista contém as informações de uma localização específica em um dicionário.

Agora, vamos utilizar a função `json_normalize()` da biblioteca `Pandas` para transformar esta lista em um `DataFrame`, onde cada linha será correspondente à cada elemento da lista:

In [30]:
tb_weather = pd.json_normalize(lista_resultados)
tb_weather


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,...,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph
0,Taboao Da Serra,Sao Paulo,Brazil,-23.57,-46.69,America/Sao_Paulo,1678823373,2023-03-14 16:49,1678823100,2023-03-14 16:45,...,0.00,83,75,25.2,77.4,9.0,5.0,5.0,8.3,13.3
1,Belem,Para,Brazil,-1.34,-48.60,America/Belem,1678823373,2023-03-14 16:49,1678823100,2023-03-14 16:45,...,0.10,84,75,33.0,91.4,10.0,6.0,5.0,1.1,1.8
2,Olinda,Pernambuco,Brazil,-7.99,-34.87,America/Recife,1678823374,2023-03-14 16:49,1678823100,2023-03-14 16:45,...,0.00,66,25,32.4,90.3,10.0,6.0,7.0,10.5,16.9
3,Lagos,Lagos,Nigeria,6.51,3.39,Africa/Lagos,1678823374,2023-03-14 20:49,1678823100,2023-03-14 20:45,...,0.00,75,25,33.8,92.9,10.0,6.0,1.0,19.2,31.0
4,East Point,Georgia,United States of America,33.69,-84.44,America/New_York,1678823374,2023-03-14 15:49,1678823100,2023-03-14 15:45,...,0.00,34,0,7.2,45.0,16.0,9.0,4.0,15.7,25.2
5,Murmansk,Murmansk,Russia,69.00,33.13,Europe/Moscow,1678823375,2023-03-14 22:49,1678823100,2023-03-14 22:45,...,0.01,79,60,-11.6,11.1,2.5,1.0,1.0,6.7,10.8


A função transformou os dicionários hierarquicos em colunas individuais: para evitar a colisão de nomes de colunas, utilizou-se a notação de `.`: por exemplo, o sub-dicionário `location`, que contém campos relativos à localização, foi armazenado nas colunas `location.name`, `location.region`, etc...

In [31]:
tb_weather.columns

Index(['location.name', 'location.region', 'location.country', 'location.lat',
       'location.lon', 'location.tz_id', 'location.localtime_epoch',
       'location.localtime', 'current.last_updated_epoch',
       'current.last_updated', 'current.temp_c', 'current.temp_f',
       'current.is_day', 'current.condition.text', 'current.condition.icon',
       'current.condition.code', 'current.wind_mph', 'current.wind_kph',
       'current.wind_degree', 'current.wind_dir', 'current.pressure_mb',
       'current.pressure_in', 'current.precip_mm', 'current.precip_in',
       'current.humidity', 'current.cloud', 'current.feelslike_c',
       'current.feelslike_f', 'current.vis_km', 'current.vis_miles',
       'current.uv', 'current.gust_mph', 'current.gust_kph'],
      dtype='object')

In [32]:
tb_weather.describe()

,location.lat,location.lon,location.localtime_epoch,current.last_updated_epoch,current.temp_c,current.temp_f,current.is_day,current.condition.code,current.wind_mph,current.wind_kph,...,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph
count,6.000000,6.000000,6.000000e+00,6.000000e+00,6.000000,6.000000,6.000000,6.0000,6.000000,6.000000,...,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.00000
mean,12.716667,-29.680000,1.678823e+09,1.678823e+09,18.333333,65.000000,0.666667,1044.5000,10.250000,16.450000,...,0.018333,70.166667,43.333333,20.000000,68.016667,9.583333,5.500000,3.833333,10.250000,16.50000
std,33.434399,41.741158,7.527727e-01,0.000000e+00,14.746751,26.544152,0.516398,103.1305,4.840558,7.810954,...,0.040208,18.882973,31.091264,18.453401,33.225979,4.294376,2.588436,2.401388,6.480664,10.43954
min,-23.570000,-84.440000,1.678823e+09,1.678823e+09,-8.000000,17.600000,0.000000,1000.0000,4.300000,6.800000,...,0.000000,34.000000,0.000000,-11.600000,11.100000,2.500000,1.000000,1.000000,1.100000,1.80000
25%,-6.327500,-48.122500,1.678823e+09,1.678823e+09,13.250000,55.850000,0.250000,1003.0000,8.100000,13.000000,...,0.000000,68.250000,25.000000,11.700000,53.100000,9.250000,5.250000,1.750000,7.100000,11.42500
50%,2.585000,-40.780000,1.678823e+09,1.678823e+09,25.000000,77.000000,1.000000,1003.0000,9.300000,14.950000,...,0.000000,77.000000,42.500000,28.800000,83.850000,10.000000,6.000000,4.500000,9.400000,15.10000
75%,26.895000,-6.175000,1.678823e+09,1.678823e+09,28.500000,83.300000,1.000000,1003.0000,11.550000,18.550000,...,0.007500,82.000000,71.250000,32.850000,91.125000,10.000000,6.000000,5.000000,14.400000,23.12500
max,69.000000,33.130000,1.678823e+09,1.678823e+09,29.000000,84.200000,1.000000,1255.0000,18.600000,29.900000,...,0.100000,84.000000,75.000000,33.800000,92.900000,16.000000,9.000000,7.000000,19.200000,31.00000


# Encapsulando APIs 

Muitas vezes as funções de uma API são encapsuladas em um **wrapper**: uma classe que contém todo o código necessário para autenticar e parametrizar queries em uma dada API. Vamos construir um **wrapper** para a nossa tarefa acima.

In [33]:
class WeatherAPI:
    '''
    Wrapper para a API https://www.weatherapi.com/docs/
    '''
    def __init__(self, token):
        '''
        Inicializa o wrapper.
        token Str: Token de identificação do usuário da API
        '''
        self.header = {'key' : token}

    def query_latlong(self, lat, long):
        '''
        Busca dados meterológicos atuais para uma localização específica
        lat Float: latitude da localização
        long Float: longitude da localização
        '''
        query = {'q' : f"{lat:.4f},{long:.4f}"}
        try:
            response = requests.request('GET', endpoint, 
                                        headers=self.header, 
                                        params=query)
            response.raise_for_status()
            return response.json()
        except HTTPError as e:
            print(e)
            return None
    

In [34]:
minha_api = WeatherAPI('bfef1b47e3b8462199b191725221801')
minha_api.query_latlong(-15, -47)

{'location': {'name': 'Odilon Guimaraes',
  'region': 'Minas Gerais',
  'country': 'Brazil',
  'lat': -15.0,
  'lon': -47.0,
  'tz_id': 'America/Sao_Paulo',
  'localtime_epoch': 1678823390,
  'localtime': '2023-03-14 16:49'},
 'current': {'last_updated_epoch': 1678823100,
  'last_updated': '2023-03-14 16:45',
  'temp_c': 26.2,
  'temp_f': 79.2,
  'is_day': 1,
  'condition': {'text': 'Cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/119.png',
   'code': 1006},
  'wind_mph': 2.5,
  'wind_kph': 4.0,
  'wind_degree': 340,
  'wind_dir': 'NNW',
  'pressure_mb': 1010.0,
  'pressure_in': 29.84,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 59,
  'cloud': 73,
  'feelslike_c': 27.3,
  'feelslike_f': 81.1,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 6.0,
  'gust_mph': 4.3,
  'gust_kph': 6.8}}